In [33]:
import csv
import json
import pandas as pd
from datetime import datetime

vendorMaster = {}
with open('/Users/patelrudra/Documents/BOOKS/Contacts.csv', mode ='r')as file:
    masterFile = csv.reader(file)    
    for vendor in masterFile:
        vendorMaster[vendor[1]] = vendor[0]
        
vendorItem = {}
with open('/Users/patelrudra/Documents/BOOKS/Item.csv', mode ='r')as file1:
    masterFile = csv.reader(file1)    
    for vendor in masterFile:
        vendorItem[vendor[1]] = vendor[0]
        
gstConfigurationData = {
    "0" : "1357403000000016427",
    "5" : "1357403000000016433",
    "12" : "1357403000000016439",
    "18" : "1357403000000016445",
    "28" : "1357403000000016451"
}
        
with open('/Users/patelrudra/Documents/Medkart/wms/credit note/R HO.csv', mode ='r')as file:

    csvFile = csv.reader(file)
    data_list = []
    null_voucher_dict = []
    data={}
    count=0
    Bill=True
    notItems=[]
    
    for entry in csvFile:
        if entry[2] == '':
            if entry[1] == 'Grand Total':
                continue;
            data1 = {
                'item_id': vendorItem[entry[1]],
                'rate':entry[12],
                "quantity":"1",
                "item_order":len(data_list)+1
             }
            
            data_list.append(data1)
        else:
            data['line_items'] = data_list
            null_voucher_dict.append(data)
            data_list=[]
            date_object = datetime.strptime(entry[0], "%d-%b-%y")
            formatted_date = date_object.strftime("%Y-%m-%d")

            data = {
                'date': formatted_date,
                'customer_id': vendorMaster[entry[1]],
                "creditnote_number" : entry[2],
#                 "gst_no":entry[5],
                 "is_draft":False,
                "is_inclusive_tax": False,
                "reference_number":entry[3],
                'reference_invoice_type':"registered",
                    'gst_reason':"sales_return",
                "adjustment": entry[21],
                "adjustment_description": "adjustment",
                
             }
            
    for data in null_voucher_dict:
        json_data = json.dumps(data, indent=4)
        with open("sample.json", "a") as outfile:
            outfile.write(json_data + '\n')  

In [34]:
null_voucher_dict.pop(0)
#print(len(null_voucher_dict))
null_voucher_dict


[{'date': '2023-04-13',
  'customer_id': '1357403000004221775',
  'creditnote_number': '230HO1IS00000166',
  'is_draft': False,
  'is_inclusive_tax': False,
  'reference_number': '230HO1IS00000166',
  'reference_invoice_type': 'registered',
  'gst_reason': 'sales_return',
  'adjustment': '0.10',
  'adjustment_description': 'adjustment',
  'line_items': [{'item_id': '1357403000004358983',
    'rate': '10828.35',
    'quantity': '1',
    'item_order': 1},
   {'item_id': '1357403000004359109',
    'rate': '584.13',
    'quantity': '1',
    'item_order': 2},
   {'item_id': '1357403000004360315',
    'rate': '3831.20',
    'quantity': '1',
    'item_order': 3}]},
 {'date': '2023-04-13',
  'customer_id': '1357403000004221775',
  'creditnote_number': '230HO1IS00000167',
  'is_draft': False,
  'is_inclusive_tax': False,
  'reference_number': '230HO1IS00000167',
  'reference_invoice_type': 'registered',
  'gst_reason': 'sales_return',
  'adjustment': '0.48',
  'adjustment_description': 'adjustm

In [35]:
import requests
import json
url = "https://www.zohoapis.in/books/v3/creditnotes?organization_id=60022053195"
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Zoho-oauthtoken 1000.f4820fd94d75e0013e4d0f37a14797a1.1712a3cb0ab4486fa15b4326125ccd86'
}
failedData = []
for data in null_voucher_dict:
    payload = json.dumps(data)


    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code >= 200 and response.status_code < 300:  # Assuming a successful response has a status code of 200
        data = response.json()
        # Process the response data as needed
        # print(data)
    else:

        print(f"API request failed with status code {response.status_code}")
        response_content = response.content.decode('utf-8')
        failedData.append(data['creditnote_number'])
        print(response_content)

print("__________________________________")
print("Finished")
print("__________________________________")
print("failed records bill number : ")
print(failedData)

API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
API request failed with status code 400
{"code":12018,"message":"The credit note# specified already exists."}
__________